### Analysis on HD-Yolo Segmentation Results

In [1]:
from nuclei_analysis import *
from time import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
input_dir = r"C:\Users\fiddl\IDC-GDC\CCG-MILD\Outputs\output_1.csv"

df = pd.read_csv(input_dir,sep=',')
headers = df['Sample_ID']
df.set_index('Sample_ID', inplace=True)
df.head()


,c_ploidy,n_ploidy,ground_truth
Sample_ID,,,
B587-TTP1,1.070084,1.097682,3.92
B588-TTP1,1.071604,1.072743,1.71
B589-TTP1,1.069502,1.089759,2.15
B58A-TTP1,1.020493,1.171277,1.84
B58B-TTP1,1.057322,1.080000,1.85


In [3]:
WGD_csv = r"G:\My Drive\ISB - Work\Texture Analysis\Clinical Data\MILD\MILD_WGD_TRUNCATED.csv"

WGD_df = pd.read_csv(WGD_csv, sep=',')
#WGD_values = WGD_df['WGD Label']
#WGD_df.set_index('Submitter ID',inplace=True)
WGD_df.head()


,Submitter ID,WGD Label,Purity,Ploidy
0,B5B1-TTP2,WGD,0.20,3.62
1,B59G-TTP1,no_WGD,0.27,2.11
2,B5B8-TTP1,no_WGD,0.22,2.02
3,B5A3-TTP1,no_WGD,0.24,1.89
4,B597-TTP1,no_WGD,0.24,1.78


In [4]:
WGD_df.sort_values(by=['Submitter ID'], inplace=True)
TBD = []
for index, label in zip(WGD_df['Submitter ID'], WGD_df['WGD Label']):
    if label != 'WGD' and label != 'no_WGD':
        TBD.append(index)

WGD_df.set_index('Submitter ID', inplace=True)
WGD_df.head()
WGD_df.drop(TBD, axis=0, inplace=True)
len(WGD_df)

142

In [5]:
TBD = []
for item in df.index:
    if item not in WGD_df.index:
        TBD.append(item)

df.drop(TBD, axis=0, inplace=True)
len(df)

142

In [6]:
WGD_values = []
for item in WGD_df['WGD Label']:
    if item == 'WGD':
        WGD_values.append(1)
    elif item == 'no_WGD':
        WGD_values.append(0)

In [7]:
RF = RandomForestClassifier()
X, y = df[['c_ploidy', 'n_ploidy']], WGD_values
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)

RF = RF.fit(X_train, y_train)
results = RF.predict(X_test)
score = accuracy_score(y_test, results)

print(score)

0.3448275862068966


In [9]:
output_dir = r"C:\Users\fiddl\IDC-GDC\CCG-MILD\Outputs"
i=0
for output in os.listdir(output_dir):
    i+=1
    input_dir = os.path.join(output_dir, output)
    df = pd.read_csv(input_dir,sep=',')
    headers = df['Sample_ID']
    df.set_index('Sample_ID', inplace=True)
    TBD = []
    for item in df.index:
        if item not in WGD_df.index:
            TBD.append(item)

    df.drop(TBD, axis=0, inplace=True)
    RF = RandomForestClassifier()
    X, y = df[['c_ploidy', 'n_ploidy']], WGD_values
    X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2)

    RF = RF.fit(X_train, y_train)
    results = RF.predict(X_test)
    score = accuracy_score(y_test, results)

    print('Output',i,':',score)   

Output 1 : 0.6206896551724138
Output 2 : 0.4827586206896552
Output 3 : 0.3793103448275862
Output 4 : 0.5172413793103449
Output 5 : 0.4482758620689655
